In [26]:
# importar librerias para el proyecto
import pandas as pd
import numpy as np
import seaborn as sbn
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.naive_bayes import GaussianNB
from sklearn import linear_model, tree
from sklearn.feature_selection import RFE
from sklearn import linear_model, tree
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.tree import DecisionTreeClassifier
from dtreeviz.trees import dtreeviz
from sklearn.model_selection import GridSearchCV
import joblib as joblib
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization, Activation
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, KFold
import keras.backend as K
#from keras.wrappers.scikit_learn import KerasRegressor
#from scikeras.wrappers import KerasClassifier


ModuleNotFoundError: No module named 'keras.wrappers'

In [2]:
# Cargamos los datos  
datos = pd.read_csv("../Datos/ObesityDataSet_raw_and_data_sinthetic.csv")
datos.head(10)

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad
0,Female,21.0,1.62,64.0,yes,no,2.0,3.0,Sometimes,no,2.0,no,0.0,1.0,no,Public_Transportation,Normal_Weight
1,Female,21.0,1.52,56.0,yes,no,3.0,3.0,Sometimes,yes,3.0,yes,3.0,0.0,Sometimes,Public_Transportation,Normal_Weight
2,Male,23.0,1.80,77.0,yes,no,2.0,3.0,Sometimes,no,2.0,no,2.0,1.0,Frequently,Public_Transportation,Normal_Weight
3,Male,27.0,1.80,87.0,no,no,3.0,3.0,Sometimes,no,2.0,no,2.0,0.0,Frequently,Walking,Overweight_Level_I
4,Male,22.0,1.78,89.8,no,no,2.0,1.0,Sometimes,no,2.0,no,0.0,0.0,Sometimes,Public_Transportation,Overweight_Level_II
5,Male,29.0,1.62,53.0,no,yes,2.0,3.0,Sometimes,no,2.0,no,0.0,0.0,Sometimes,Automobile,Normal_Weight
6,Female,23.0,1.50,55.0,yes,yes,3.0,3.0,Sometimes,no,2.0,no,1.0,0.0,Sometimes,Motorbike,Normal_Weight
7,Male,22.0,1.64,53.0,no,no,2.0,3.0,Sometimes,no,2.0,no,3.0,0.0,Sometimes,Public_Transportation,Normal_Weight
8,Male,24.0,1.78,64.0,yes,yes,3.0,3.0,Sometimes,no,2.0,no,1.0,1.0,Frequently,Public_Transportation,Normal_Weight
9,Male,22.0,1.72,68.0,yes,yes,2.0,3.0,Sometimes,no,2.0,no,1.0,1.0,no,Public_Transportation,Normal_Weight


In [6]:
datos.describe()


,Age,Height,Weight,FCVC,NCP,CH2O,FAF,TUE
count,2111.000000,2111.000000,2111.000000,2111.000000,2111.000000,2111.000000,2111.000000,2111.000000
mean,24.312600,1.701677,86.586058,2.419043,2.685628,2.008011,1.010298,0.657866
std,6.345968,0.093305,26.191172,0.533927,0.778039,0.612953,0.850592,0.608927
min,14.000000,1.450000,39.000000,1.000000,1.000000,1.000000,0.000000,0.000000
25%,19.947192,1.630000,65.473343,2.000000,2.658738,1.584812,0.124505,0.000000
50%,22.777890,1.700499,83.000000,2.385502,3.000000,2.000000,1.000000,0.625350
75%,26.000000,1.768464,107.430682,3.000000,3.000000,2.477420,1.666678,1.000000
max,61.000000,1.980000,173.000000,3.000000,4.000000,3.000000,3.000000,2.000000


In [7]:
datos['CALC'].unique()

array(['no', 'Sometimes', 'Frequently', 'Always'], dtype=object)

In [3]:
rng = np.random.RandomState(seed=30) # Algo pasa con la semilla pero en X_train la variable CAEC queda sin always, por eso al ajustar el modelo no encuentra la categoría
categorical= ["Gender","family_history_with_overweight", "FAVC","SMOKE", "SCC","CAEC","CALC",'MTRANS']
categorical_gender =["Gender"]
categorical_twocat = ["family_history_with_overweight", "FAVC","SMOKE", "SCC"]
categorical_fourcat= ["CAEC","CALC"]
categorical_multicat=['MTRANS']
cat_2=['no','yes']
cat_g=['Female', 'Male']
cat_4=['no', 'Sometimes', 'Frequently', 'Always']
cat_m=['Public_Transportation', 'Walking', 'Automobile', 'Motorbike','Bike']
numerical_columns = ["Age", "FCVC", "NCP", "CH2O", "FAF","TUE"]
X = datos[numerical_columns + categorical_gender+categorical_twocat+categorical_fourcat+categorical_multicat]
y = datos["NObeyesdad"]
y = pd.DataFrame(datos['NObeyesdad'].map({'Normal_Weight': 0, 'Insufficient_Weight': 0, 'Overweight_Level_I':1,'Overweight_Level_II':1, 'Obesity_Type_I':2,'Obesity_Type_II':2, 'Obesity_Type_III':2}))
yname = ["NObeyesdad"]
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=54)

In [5]:
X_test['CAEC'].unique()

array(['Sometimes', 'Frequently', 'no', 'Always'], dtype=object)

Preprocessing

In [23]:
# Se crea un pipeline para codificar los datos
categorical_encoder = OneHotEncoder(handle_unknown='ignore') # Revisar si es valido el ignore... Sin el ignore me sale el mismo error que te salía del 'Always'
#categorical_encoder1 = OneHotEncoder(categories=[['no', 'yes'], [0, 1]])
#categorical_encoder2 = OneHotEncoder(categories=[['Female', 'Male'], [1, 0]],handle_unknown='ignore')
#categorical_encoder3 = OneHotEncoder(categories=[['no', 'Sometimes', 'Frequently', 'Always'],[0,1,2,3]],handle_unknown='ignore')
#categorical_encoder4 = OneHotEncoder(categories=[['Public_Transportation', 'Walking', 'Automobile', 'Motorbike','Bike'],[0,1,2,3,4]],handle_unknown='ignore')
numerical_pipe = StandardScaler()

preprocessing = ColumnTransformer(
    [
        ("codificacion de categoricas",categorical_encoder, categorical),
        #("codificacion de categoricas1", categorical_encoder1, categorical_twocat),
        #("codificacion de categoricas2", categorical_encoder2, categorical_gender),
        #("codificacion de categoricas3", categorical_encoder3, categorical_fourcat),
        #("codificacion de categoricas4", categorical_encoder4, categorical_multicat),
        ("Transformación de numéricas", numerical_pipe, numerical_columns)
    ],
    verbose_feature_names_out=False,
)


Arbol de decisión

In [7]:
arbol = DecisionTreeClassifier(random_state=30)
modelo_arbol = Pipeline(
    [
    ("preprocesamiento", preprocessing),
    ("Classifier", arbol),  
    ]
)

In [8]:
k_fold = StratifiedKFold(5, shuffle=True, random_state=30)
param_grid = {"Classifier__criterion": ['entropy', 'gini','log_loss'],
              "Classifier__max_depth": range(3, 5),
              "Classifier__min_samples_leaf": range(2, 10)}

best_tree = GridSearchCV(modelo_arbol, param_grid, cv=k_fold, scoring='accuracy', n_jobs=-1)
best_tree.fit(X_train, y_train)

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=30, shuffle=True),
             estimator=Pipeline(steps=[('preprocesamiento',
                                        ColumnTransformer(transformers=[('codificacion '
                                                                         'de '
                                                                         'categoricas',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                                         ['Gender',
                                                                          'family_history_with_overweight',
                                                                          'FAVC',
                                                                          'SMOKE',
                                                                          'SCC',
                                                                          'CAEC',
                                                                          'CALC',
                                                                          'MTRANS'])],
                                                          verbose_feature_names_out=False)),
                                       ('Classifier',
                                        DecisionTreeClassifier(random_state=30))]),
             n_jobs=-1,
             param_grid={'Classifier__criterion': ['entropy', 'gini',
                                                   'log_loss'],
                         'Classifier__max_depth': range(3, 5),
                         'Classifier__min_samples_leaf': range(2, 10)},
             scoring='accuracy')

In [9]:
best_tree.best_params_

{'Classifier__criterion': 'gini',
 'Classifier__max_depth': 4,
 'Classifier__min_samples_leaf': 8}

In [10]:
print(best_tree.best_score_, best_tree.score(X_test,y_test))

0.6829034061414367 0.6761363636363636


Random Forest

In [13]:
# Random Forest
modelo=RandomForestClassifier(random_state=30)
modelo_rf = Pipeline(
    [
    ("preprocesamiento", preprocessing),
    ("clasificador", modelo),
    ]
)

#mejor_gnb= GridSearchCV(modelo_train, param_grid, scoring='accuracy')
#joblib.dump(mejor_gnb,"bm_gnb.gz")

In [14]:
k_fold = StratifiedKFold(5, shuffle=True, random_state=30)
param_grid = {'clasificador__criterion': ['entropy', 'log_loss', 'gini'],
              'clasificador__max_depth': range(3, 10),
              'clasificador__min_samples_leaf': range(2, 10)}
#param_grid ={}
best_rf = GridSearchCV(modelo_rf, param_grid, scoring='accuracy', n_jobs=-1)
best_rf.fit(X_train, y_train)

c:\Users\CARLOS MARIO\Anaconda3\envs\tensor_310\lib\site-packages\sklearn\base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


GridSearchCV(estimator=Pipeline(steps=[('preprocesamiento',
                                        ColumnTransformer(transformers=[('codificacion '
                                                                         'de '
                                                                         'categoricas',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                                         ['Gender',
                                                                          'family_history_with_overweight',
                                                                          'FAVC',
                                                                          'SMOKE',
                                                                          'SCC',
                                                                          'CAEC',
                                                                          'CALC',
                                                                          'MTRANS'])],
                                                          verbose_feature_names_out=False)),
                                       ('clasificador',
                                        RandomForestClassifier(random_state=30))]),
             n_jobs=-1,
             param_grid={'clasificador__criterion': ['entropy', 'log_loss',
                                                     'gini'],
                         'clasificador__max_depth': range(3, 10),
                         'clasificador__min_samples_leaf': range(2, 10)},
             scoring='accuracy')

In [15]:
best_rf.best_params_

{'clasificador__criterion': 'entropy',
 'clasificador__max_depth': 8,
 'clasificador__min_samples_leaf': 2}

In [16]:
print(best_rf.best_score_, best_rf.score(X_test,y_test))

0.7277342970091442 0.7196969696969697


Naive Bayes

In [17]:
gnb=GaussianNB()
modelo_GNB = Pipeline(
    [
    ("preprocesamiento", preprocessing),
    ("Classifier", gnb),  
    ]
)

k_fold = StratifiedKFold(5, shuffle=True, random_state=54)
param_grid = {}
#param_grid ={}
best_GNB = GridSearchCV(modelo_GNB, param_grid, scoring='accuracy', n_jobs=-1)
best_GNB.fit(X_train, y_train)


c:\Users\CARLOS MARIO\Anaconda3\envs\tensor_310\lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GridSearchCV(estimator=Pipeline(steps=[('preprocesamiento',
                                        ColumnTransformer(transformers=[('codificacion '
                                                                         'de '
                                                                         'categoricas',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                                         ['Gender',
                                                                          'family_history_with_overweight',
                                                                          'FAVC',
                                                                          'SMOKE',
                                                                          'SCC',
                                                                          'CAEC',
                                                                          'CALC',
                                                                          'MTRANS'])],
                                                          verbose_feature_names_out=False)),
                                       ('Classifier', GaussianNB())]),
             n_jobs=-1, param_grid={}, scoring='accuracy')

In [19]:
print(best_GNB.best_score_, best_GNB.score(X_test,y_test))

0.6556802300043925 0.6723484848484849


In [25]:
X_train.head()

,Age,FCVC,NCP,CH2O,FAF,TUE,Gender,family_history_with_overweight,FAVC,SMOKE,SCC,CAEC,CALC,MTRANS
1936,22.777890,3.000000,3.000000,1.910390,0.217455,0.775820,Female,yes,yes,no,no,Sometimes,Sometimes,Public_Transportation
384,18.000000,3.000000,1.000000,1.000000,0.000000,2.000000,Female,yes,yes,no,yes,Sometimes,no,Walking
1563,33.553360,2.576449,2.419656,1.000000,1.982379,0.000000,Male,yes,yes,no,no,Sometimes,no,Public_Transportation
618,21.478496,3.000000,3.179995,1.910378,0.480614,0.625079,Female,yes,yes,no,no,Frequently,no,Public_Transportation
490,24.000000,3.000000,3.000000,2.000000,1.000000,0.000000,Male,no,yes,no,yes,no,Sometimes,Public_Transportation


Redes Neuronales

In [31]:
#def neural_network(optimizer='adagrad',
#                  kernel_initializer='glorot_uniform', 
#                  dropout=0.2):
#    model=Sequential()
#    model.add(Dense(64,activation='relu',kernel_initializer=kernel_initializer))
#    model.add(Dropout(dropout))
#    model.add(Dense(1,activation='sigmoid',kernel_initializer=kernel_initializer))
#    model.compile(loss='binary_crossentropy',optimizer=optimizer, metrics=['accuracy'])
#    return model

red = Sequential()
red.add(Dense(14, input_shape=(28,) ,activation='softmax'))
red.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
#modelo_n=KerasRegressor(build_fn=neural_network, verbose=0)

modelo_red = Pipeline(
    [
    ("preprocesamiento", preprocessing),
    ("Classifier", red),  
    ]
)

#k_fold = StratifiedKFold(5, shuffle=True, random_state=54)
#param_grid = {'optimizer': ['adam', 'sgd'],
#              'units': [32, 64, 128]}
#param_grid ={}
#best_Red = GridSearchCV(modelo_red, param_grid, scoring='accuracy', n_jobs=-1)
modelo_red.fit(X_train, y_train)


ValueError: in user code:

    File "c:\Users\CARLOS MARIO\Anaconda3\envs\tensor_310\lib\site-packages\keras\src\engine\training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\CARLOS MARIO\Anaconda3\envs\tensor_310\lib\site-packages\keras\src\engine\training.py", line 1384, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\CARLOS MARIO\Anaconda3\envs\tensor_310\lib\site-packages\keras\src\engine\training.py", line 1373, in run_step  **
        outputs = model.train_step(data)
    File "c:\Users\CARLOS MARIO\Anaconda3\envs\tensor_310\lib\site-packages\keras\src\engine\training.py", line 1151, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "c:\Users\CARLOS MARIO\Anaconda3\envs\tensor_310\lib\site-packages\keras\src\engine\training.py", line 1209, in compute_loss
        return self.compiled_loss(
    File "c:\Users\CARLOS MARIO\Anaconda

Fin del código

In [33]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from scikeras.wrappers import KerasClassifier, KerasRegressor


model = Sequential()
model.add(Dense(units=4, input_shape=(2,), activation='sigmoid'))
model.add(Dense(units=4,  activation='sigmoid'))
model.add(Dense(units=1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Construye el Pipeline
MO = Pipeline([
    ('preprocessing', preprocessing),
    ('Classifier', model)
])

# Define los parámetros a ajustar
param_grid = {
    'optimizer': ['adam', 'sgd'],
    'units': [32, 64, 128],
    'activation': ['relu', 'tanh', 'sigmoid']
}

# Configura la búsqueda de hiperparámetros utilizando GridSearchCV
grid = GridSearchCV(estimator=MO, param_grid=param_grid, cv=3, scoring='accuracy')
grid_result = grid.fit(X_train, y_train)

# Muestra los resultados
print(f"Mejores parámetros: {grid_result.best_params_}")
print(f"Precisión del modelo: {grid_result.best_score_}")


INFO:tensorflow:Assets written to: C:\Users\IP5-14~1\AppData\Local\Temp\tmpqzlrqavu\assets


INFO:tensorflow:Assets written to: C:\Users\IP5-14~1\AppData\Local\Temp\tmpqzlrqavu\assets


INFO:tensorflow:Assets written to: C:\Users\IP5-14~1\AppData\Local\Temp\tmpis05tm53\assets


INFO:tensorflow:Assets written to: C:\Users\IP5-14~1\AppData\Local\Temp\tmpis05tm53\assets


ValueError: Invalid parameter 'activation' for estimator Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('codificacion de categoricas',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['Gender',
                                                   'family_history_with_overweight',
                                                   'FAVC', 'SMOKE', 'SCC',
                                                   'CAEC', 'CALC', 'MTRANS'])],
                                   verbose_feature_names_out=False)),
                ('Classifier',
                 <keras.src.engine.sequential.Sequential object at 0x000001813F9BB8B0>)]). Valid parameters are: ['memory', 'steps', 'verbose'].

In [23]:
KerasClassifier.get_params().keys()

TypeError: BaseEstimator.get_params() missing 1 required positional argument: 'self'

In [26]:
gnb2=joblib.load('bm_gnb.gz')
gnb2.fit(X_train, y_train)
#confusion_matrix(y_test, gnb.fit(X_test))
gnb2.best_params_

ValueError: 
All the 5 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\CARLOS MARIO\Anaconda3\envs\tensor_310\lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\CARLOS MARIO\Anaconda3\envs\tensor_310\lib\site-packages\sklearn\base.py", line 1474, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "c:\Users\CARLOS MARIO\Anaconda3\envs\tensor_310\lib\site-packages\sklearn\pipeline.py", line 471, in fit
    Xt = self._fit(X, y, routed_params)
  File "c:\Users\CARLOS MARIO\Anaconda3\envs\tensor_310\lib\site-packages\sklearn\pipeline.py", line 408, in _fit
    X, fitted_transformer

In [ ]:
# modelo_logistico= LogisticRegression()
gnb=GaussianNB()
# arbol=DecisionTreeClassifier(criterion='gini', min_samples_split=100, min_samples_leaf=50)
random_forest = RandomForestClassifier(random_state=42)

def red():
    modelo1 = Sequential()
    modelo1.add(Dense(units=1, input_shape=(16,), activation='sigmoid'))
    modelo1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return modelo1
red1 = red()

modelos=[modelo_logistico,gnb,arbol,random_forest, red()]
resultados=[]
for modelo in modelos:
    nombre_modelo=f'{modelo}.joblib'
    flujo = Pipeline(
        [
            ("Preprocesamiento", preprocessing),
            ("Clasificador", modelo),
        ]
    )
    joblib.dump(flujo, nombre_modelo)
    salida=joblib.dump(flujo, nombre_modelo)
    resultados.append(salida)

from sklearn.metrics import classification_report
#print(classification_report(y_test, y_pred))

In [ ]:
resultados

In [ ]:

rf=joblib.load('r_RandomForestClassifier(random_state=42).joblib')
confusion_matrix(y_test, rf.predict(X_test))
rf



In [ ]:
rf.predict(X_test)

In [ ]:
log=joblib.load('r_LogisticRegression().joblib')
confusion_matrix(y_test, log.predict(X_test))

In [ ]:
nb=joblib.load('r_GaussianNB().joblib')
confusion_matrix(y_test, nb.predict(X_test))

In [ ]:
log.fit(X_train,y_train)